In [1]:
import os


In [2]:
import torch
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA是否可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA版本: {torch.version.cuda}")
    print(f"当前GPU设备: {torch.cuda.get_device_name(0)}")

PyTorch版本: 2.6.0+cu118
CUDA是否可用: True
CUDA版本: 11.8
当前GPU设备: NVIDIA GeForce RTX 3090


In [ ]:
import soundfile as sf
from modelscope import snapshot_download
from modelscope import Qwen2_5OmniForConditionalGeneration, Qwen2_5OmniProcessor
from qwen_omni_utils import process_mm_info
import torch

/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# model_dir = snapshot_download(
#     "Qwen/Qwen2.5-Omni-7B", 
#     cache_dir="./cache/modelscope"
# )
model_dir = snapshot_download(
    'Qwen/Qwen2.5-Omni-3B',
    cache_dir="./cache/modelscope"
)

2025-12-04 15:01:50,092 - modelscope - INFO - Target directory already exists, skipping creation.


In [7]:
model = Qwen2_5OmniForConditionalGeneration.from_pretrained(
    model_dir,
    device_map="cuda:1",
    dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)
model.disable_talker()
processor = Qwen2_5OmniProcessor.from_pretrained(model_dir)

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.46s/it]


In [8]:
print("模型设备分布:", model.hf_device_map)

模型设备分布: {'': device(type='cuda', index=1)}


In [9]:
conversation = [
    {
        "role": "system",
        "content": [
            {"type": "text", "text": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech."}
        ],
    },
    {
        "role": "user",
        "content": [
            {"type": "video", "video": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen2.5-Omni/draw.mp4"},
            {"type": "text", "text": "Could you detail events during different time segments? Format strictly:\nFrom xx to xx, event1.\nFrom xx to xx, event2.\n..."}
        ],
    },
]

USE_AUDIO_IN_VIDEO = True

text = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
audios, images, videos = process_mm_info(conversation, use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = processor(text=text, audio=audios, images=images, videos=videos, return_tensors="pt", padding=True, use_audio_in_video=USE_AUDIO_IN_VIDEO)
inputs = inputs.to(model.device).to(model.dtype)

/home/liuchi/anaconda3/envs/qwen_lora/lib/python3.10/site-packages/librosa/core/audio.py:172: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
qwen-vl-utils using torchvision to read video.


In [10]:
text_ids = model.generate(**inputs, use_audio_in_video=USE_AUDIO_IN_VIDEO, return_audio=False)

text = processor.batch_decode(text_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(text)

['system\nYou are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.\nuser\nCould you detail events during different time segments? Format strictly:\nFrom xx to xx, event1.\nFrom xx to xx, event2.\n...\nassistant\nFrom 0.00 to 1.00, the person is holding a tablet with a guitar drawing on it.\nFrom 1.00 to 2.00, the person is using a pen to draw on the tablet.\nFrom 2.00 to 3.00, the person is holding the tablet with one hand and using the pen with the other hand.\nFrom 3.00 to 4.00, the person is using the pen to draw on the tablet.\nFrom 4.00 to 5.00, the person is holding the tablet with one hand and using the pen with the other hand.\nFrom 5.00 to 6.00, the person is using the pen to draw on the tablet.\nFrom 6.00 to 7.00, the person is holding the tablet with one hand and using the pen with the other hand.\nFrom 7.00 to 8.00, the person is using the pen to draw on the tablet.\nFro

In [11]:
print("模型设备分布:", model.hf_device_map)

模型设备分布: {'': device(type='cuda', index=1)}
